In [1]:
from mcts_model import AlphaZeroNN, ReplayBuffer, MCTS_Policy, Trainer, generate_lbfs_start_states, CONFIG
from mcts_worker import run_episode_worker

In [4]:
lbfs_start_states = generate_lbfs_start_states(num_states=20000, warmup=100000, separation=5000)

--- Generating 20000 Warm Start States using LBFS ---
Warming up for 100000 time units...
  Captured 50/20000 states...
  Captured 100/20000 states...
  Captured 150/20000 states...
  Captured 200/20000 states...
  Captured 250/20000 states...
  Captured 300/20000 states...
  Captured 350/20000 states...
  Captured 400/20000 states...
  Captured 450/20000 states...
  Captured 500/20000 states...
  Captured 550/20000 states...
  Captured 600/20000 states...
  Captured 650/20000 states...
  Captured 700/20000 states...
  Captured 750/20000 states...
  Captured 800/20000 states...
  Captured 850/20000 states...
  Captured 900/20000 states...
  Captured 950/20000 states...
  Captured 1000/20000 states...
  Captured 1050/20000 states...
  Captured 1100/20000 states...
  Captured 1150/20000 states...
  Captured 1200/20000 states...
  Captured 1250/20000 states...
  Captured 1300/20000 states...
  Captured 1350/20000 states...
  Captured 1400/20000 states...
  Captured 1450/20000 states...
  

In [3]:
import torch
from mcts_model import AlphaZeroNN, MCTS_Policy, CONFIG

# Define the dimensions exactly as they were during training
# (For your L=2 network, state_size=10 and action_space_size=7)
state_size = CONFIG["MAX_QUEUES_STATE"]

# A safe way to get the action size without guessing is to init a dummy policy
temp_policy = MCTS_Policy(None, CONFIG) 
action_space_size = len(temp_policy.master_action_list)

# 1. Create a fresh, empty model (random weights)
loaded_model = AlphaZeroNN(state_size, action_space_size)

# 2. Load the dictionary of numbers from the file
path_to_file = "mcts_alphazero_model.pth" # Change to your filename
weights = torch.load(path_to_file, map_location=CONFIG["device"])

# 3. Pour the weights into the model
loaded_model.load_state_dict(weights)

# 4. Set to Evaluation Mode (Important for Inference)
loaded_model.eval()
loaded_model.to(CONFIG["device"])

print("Model loaded successfully!")

Model loaded successfully!


/var/folders/v7/k9ypfzq16258x69j7s6yh06w0000gn/T/ipykernel_39508/2388792479.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(path_to_file, map_locat

In [ ]:
trainer = Trainer(CONFIG)
trainer.run_training_loop_parallel(num_workers=4)


--- PARALLEL TRAINING: 100 loops x 30 eps ---

--- LOOP 1/100 ---


In [4]:
trainer.save_model()

Model saved to mcts_alphazero_model.pth
